In [1]:
# Email from Gianna and JT 20200318: BL Store Addresses 
# 2:06 PM

import pandas as pd
import json
import numpy as np
from haversine import haversine
import datetime
import os
import glob

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))
print(os.getcwd())
print(datetime.datetime.now())

/home/jian/Projects/Big_Lots/Analysis/2020_Q1/plannder_requests/Gianna_zips_by_store_by_day_20200318
2020-03-18 15:35:42.575538


In [2]:
df_store_address=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200301-134228-900.txt",
                              sep="|",dtype=str)
df_store_address.head(2)

,location_id,location_desc,open_dt,close_dt,address_line_1,address_line_2,city_nm,state_nm,zip_cd,longitude_meas,latitude_meas,sales_square_feet,furniture_sales_square_feet
0,3,PEACH ORCHARD PLAZA,1988-10-31,NaN,2708 PEACH ORCHARD RD,NaN,AUGUSTA,GA,30906-2408,-000000000082.018921,000000000033.421570,00000000026501.0000,00000000005098.0000
1,30,SOUTHWOOD PLZ S/C,1984-04-05,NaN,818 S MAIN ST,NaN,BOWLING GREEN,OH,43402-4601,-000000000083.650909,000000000041.363132,00000000021322.0000,00000000003271.0000


In [3]:
df_store_address=df_store_address[['location_id','address_line_1','address_line_2','city_nm','state_nm',
                                   'zip_cd','latitude_meas','longitude_meas']]
df_store_address['latitude_meas']=df_store_address['latitude_meas'].astype(float)
df_store_address['longitude_meas']=df_store_address['longitude_meas'].astype(float)
df_store_address['zip_cd']=df_store_address['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
df_store_address=df_store_address.rename(columns={"zip_cd":"store_zip"})

In [4]:
list_excel_file=glob.glob("./input_store_list/*.xlsx")
print(len(list_excel_file))

df_all_wanted_stores={}
for file in list_excel_file:
    date=os.path.basename(file).lower().split(" del")[0]
    excel_file=pd.ExcelFile(file)
    if len(excel_file.sheet_names)>1:
        print(file,"multiple sheets")
    else:
        df=excel_file.parse(excel_file.sheet_names[0],dtype=str)
        print(df.columns.tolist())
        df['Store #']=df['Store #'].apply(lambda x: str(int(x.strip())))
        df_all_wanted_stores.update({date:df})

7
['Store #', 'Destination\nCity', 'State', 'Des\nZip\nCode', 'Delivery\nAppointment']
['Store #', 'Destination\nCity', 'State', 'Des\nZip\nCode', 'Delivery\nAppointment']
['Store #', 'Destination\nCity', 'State', 'Des\nZip\nCode', 'Delivery\nAppointment']
['Store #', 'Destination\nCity', 'State', 'Des\nZip\nCode', 'Delivery\nAppointment']
['Store #', 'Destination\nCity', 'State', 'Des\nZip\nCode', 'Delivery\nAppointment']
['Store #', 'Destination\nCity', 'State', 'Des\nZip\nCode', 'Delivery\nAppointment']
['Store #', 'Destination\nCity', 'State', 'Des\nZip\nCode', 'Delivery\nAppointment']


In [5]:
list_all_wanted_stores=[]
for v_df in df_all_wanted_stores.values():
    store_list=v_df['Store #'].unique().tolist()
    list_all_wanted_stores.extend(store_list)
list_all_wanted_stores=list(set(list_all_wanted_stores))

print(len(list_all_wanted_stores))

571


In [6]:
# all currently open stores
[x for x in list_all_wanted_stores if x not in df_store_address['location_id'].tolist()]


[]

In [7]:
TA_zips=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
TA_zips=TA_zips.parse("view_by_store",dtype=str)
TA_zips.head(2)

,TA_num,ta_name,location_id,address_line_1,address_line_2,zip_cd,city_nm,state_nm,latitude_meas,longitude_meas,DMA,CTY,all_trans_P_zips,all_trans_S_zips,zips_in_10,trans_P_zips_70_within_TA,trans_S_zips_70_within_TA
0,1,multiple_1,1949,3178 LAVON DR,NaN,75040,GARLAND,TX,32.945015,-96.619194,{'DALLAS-FT. WORTH'},{'DALLAS'},"['75040', '75044', '75048', '75098']","['75089', '75042', '75043', '75041', '75088', ...",[],"['75040', '75098', '75044', '75048']","['75088', '75042', '75082', '75043', '75089', ..."
1,1,multiple_1,1255,1418 W MOORE AVE,NaN,75160,TERRELL,TX,32.738772,-96.299877,{'DALLAS-FT. WORTH'},"{'HUNT', 'KAUFMAN', 'ROCKWALL'}","['75160', '75126', '75161']","['75142', '75169', '75474', '75143', '75114', ...",[],"['75126', '75161', '75160']","['75142', '75169', '75150', '75117', '75143', ..."


In [8]:
df_temporary=TA_zips[['location_id','trans_P_zips_70_within_TA','trans_S_zips_70_within_TA','zips_in_10']]
df_zip_by_store=pd.DataFrame()

for ind,row in df_temporary.iterrows():
    location_id=row['location_id']
    P_zips=eval(row['trans_P_zips_70_within_TA'])
    S_zips=eval(row['trans_S_zips_70_within_TA'])
    zip_10=eval(row['zips_in_10'])
    

    df_P=pd.DataFrame(zip([location_id]*len(P_zips),P_zips))
    if len(df_P)>0:
        df_P.columns=['location_id','zip_cd']
        df_P['zip_type']="P"
        
    df_S=pd.DataFrame(zip([location_id]*len(S_zips),S_zips))
    if len(df_S)>0:
        df_S.columns=['location_id','zip_cd']
        df_S['zip_type']="S"
    
    df_10=pd.DataFrame(zip([location_id]*len(zip_10),zip_10))
    if len(df_10)>0:
        df_10.columns=['location_id','zip_cd']
        df_10['zip_type']="zip_10"
    
    df_zip_by_store=df_zip_by_store.append(df_P).append(df_S).append(df_10)

In [9]:
df_existing_stores=df_zip_by_store[df_zip_by_store['location_id'].isin(list_all_wanted_stores)]
print(df_existing_stores.shape)
print(df_existing_stores['location_id'].nunique())
df_existing_stores.head(2)

(8986, 3)
558


,location_id,zip_cd,zip_type
0,1949,75040,P
1,1949,75098,P


In [10]:
list_new_stores=[x for x in list_all_wanted_stores if x not in df_zip_by_store['location_id'].tolist()]
len(list_new_stores)

13

In [11]:
df_new_stores=df_store_address[df_store_address['location_id'].isin(list_new_stores)]
print(df_new_stores.shape)
df_10_zips_all_new_stores=pd.DataFrame()
for ind, row in df_new_stores.iterrows():
    location_id=row['location_id']
    store_location=(row['latitude_meas'],row['longitude_meas'])
    for zip_cd,v in zip_centers.items():
        dist=haversine(store_location,v,unit="mi")
        if dist<=10:
            # print(location_id,zip_cd)
            df=pd.DataFrame({"location_id":[location_id],"zip_cd":[zip_cd],"dist":[dist]})
            df_10_zips_all_new_stores=df_10_zips_all_new_stores.append(df)
df_10_zips_all_new_stores['zip_type']="zip_10"
print(df_10_zips_all_new_stores.shape)
del df_10_zips_all_new_stores['dist']

df_10_zips_all_new_stores.head(3)

(13, 8)
(292, 4)


,location_id,zip_cd,zip_type
0,4702,79968,zip_10
0,4702,79901,zip_10
0,4702,79902,zip_10


In [12]:
df_all_wanted_store_zips=df_existing_stores.append(df_10_zips_all_new_stores)
print(df_all_wanted_store_zips.shape,df_all_wanted_store_zips['location_id'].nunique())
df_all_wanted_store_zips=pd.merge(df_all_wanted_store_zips,df_store_address,on="location_id",how="left")
df_all_wanted_store_zips.head(3)

(9278, 3) 571


,location_id,zip_cd,zip_type,address_line_1,address_line_2,city_nm,state_nm,store_zip,latitude_meas,longitude_meas
0,1949,75040,P,3178 LAVON DR,NaN,GARLAND,TX,75040,32.945015,-96.619194
1,1949,75098,P,3178 LAVON DR,NaN,GARLAND,TX,75040,32.945015,-96.619194
2,1949,75044,P,3178 LAVON DR,NaN,GARLAND,TX,75040,32.945015,-96.619194


In [13]:
df_qc_store_address=df_store_address.copy()
df_qc_store_address=df_qc_store_address[df_qc_store_address['location_id'].isin(list_all_wanted_stores)]
df_agg_from_Gianna=pd.DataFrame()
for v in df_all_wanted_stores.values():
    df_agg_from_Gianna=df_agg_from_Gianna.append(v)
del df_agg_from_Gianna['Delivery\nAppointment']
df_agg_from_Gianna=df_agg_from_Gianna.drop_duplicates()
df_agg_from_Gianna.shape

(571, 4)

In [14]:
df_qc_store_address=pd.merge(df_agg_from_Gianna,df_qc_store_address,left_on="Store #",right_on="location_id",how='inner')
df_qc_store_address.shape

(571, 12)

In [15]:
writer=pd.ExcelWriter("./BL_zips_by_dates_delivery_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_all_wanted_store_zips.to_excel(writer,"all_stores_and_zips",index=False)
df_qc_store_address.to_excel(writer,"qc_address",index=False)

dict_unique_zip_by_date={}
for k,v in df_all_wanted_stores.items():
    df=v.rename(columns={"Store #":"location_id"})
    df=pd.merge(df,df_all_wanted_store_zips,on="location_id",how="left")
    df.to_excel(writer,k,index=False)
    dict_unique_zip_by_date.update({k:df['zip_cd'].unique().tolist()})
    
df_unique_zips_by_date=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in dict_unique_zip_by_date.items() ]))
df_unique_zips_by_date.to_excel(writer,"unique_zip_by_date",index=False)
writer.save()

In [16]:
df_unique_zips_by_date.shape

(2229, 7)